In [1]:
import matplotlib.image as image 
import matplotlib.pyplot as plt
import numpy as np
import cv2

from PIL import Image
from math import log10, sqrt
from skimage import exposure

In [5]:
# Function to print 1 image
def printImage(image, title):

    fig, ax = plt.subplots(figsize=(10,10)) 

    ax.imshow(image) 
    ax.axis('off') 

    plt.title(title, fontweight ="bold",fontsize=15) 
    plt.show()
    
# Function to print 2 image
def print2Image(image1, title1, image2, title2):
    
    fig, ax = plt.subplots(1, 2, figsize=(15,15)) 

    ax[0].imshow(image1) 
    ax[1].imshow(image2)
    ax[0].set_title(title1, fontweight ="bold",fontsize=15) 
    ax[1].set_title(title2, fontweight ="bold",fontsize=15) 
    ax[0].axis('off') 
    ax[1].axis('off')

    plt.show()
    
# Function to print 3 image
def print3Image(image1, title1, image2, title2, image3, title3):
    
    fig, ax = plt.subplots(1, 3, figsize=(15,15)) 

    ax[0].imshow(image1) 
    ax[1].imshow(image2)
    ax[2].imshow(image3)
    ax[0].set_title(title1, fontweight ="bold",fontsize=15) 
    ax[1].set_title(title2, fontweight ="bold",fontsize=15) 
    ax[2].set_title(title3, fontweight ="bold",fontsize=15) 
    ax[0].axis('off') 
    ax[1].axis('off')
    ax[2].axis('off')

    plt.show()
    
# Function to change RGB to HSV
def rgbToHsv(rgbImg):
    
    hsvImg = rgbImg.copy()
    hsvImg = cv2.cvtColor(rgbImg, cv2.COLOR_RGB2HSV)
    
    print2Image(rgbImg, "RGB Image", hsvImg, "HSV Image")
    
    return hsvImg
    
# Function to apply Histogram Equation on HSV Image
def hist_eq(img):
    img_hist = img.copy()
    i_img = img_hist[:,:,2]
    i_img = cv2.equalizeHist(i_img)
    img_hist = np.dstack((img_hist[:,:,0],img_hist[:,:,1],i_img))
    img_hist = cv2.cvtColor(img_hist, cv2.COLOR_HSV2RGB)
    
    return img_hist

# Function to apply Contrast Stretching on HSV Image
def contr_stretch(img):
    p2 = np.percentile(img, 2)
    p98 = np.percentile(img, 98)
    img_contr = exposure.rescale_intensity(img, in_range=(p2, p98))
    img_contr = cv2.cvtColor(img_contr, cv2.COLOR_HSV2RGB)
    
    return img_contr

# Function to apply CLAHE on HSV Image
def clahe(img):
    clahe_enh = cv2.createCLAHE(clipLimit =2.0, tileGridSize=(8,8))
    img_split = cv2.split(img)
    img_split[0] = clahe_enh.apply(img_split[0])
    cl_img = cv2.merge(img_split)
    cl_img = cv2.cvtColor(cl_img, cv2.COLOR_HSV2RGB)
    
    return cl_img

# Applying contrast enhancement into HSV Image
def contrast_enhancement(img):
    img_hist = hist_eq(img)
    img_contr = contr_stretch(img)
    img_clahe = clahe(img)
    
    print3Image(img_hist, "Histogram Equation", img_contr, "Contrast Stretching", img_clahe, "CLAHE")
    
    return img_hist, img_contr, img_clahe
    
# Applying mean, median, and gaussian filtering
def image_filtering(img):
    kernel = np.array([[1,2,1],[2,4,2],[1,2,1]])/16

    mean_img = cv2.filter2D(img,-1,kernel)
    med_img = cv2.medianBlur(img, 3)
    gaussian_img = cv2.GaussianBlur(img,(5,5),0)

    print3Image(mean_img, "Mean Filter", med_img, "Median Filter", gaussian_img, "Gaussian Filter")
    
    return mean_img, med_img, gaussian_img
    
# Applying image filter from [[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]] matrix
def outline_image_filter(img):
    edge_detection = np.array([[ -1, -1, -1], 
                        [ -1, 8, -1], 
                        [ -1, -1, -1]])

    # Filter the image using filter2D, which has inputs: (grayscale image, bit-depth, kernel)  
    filtered_image_edges = cv2.filter2D(img, -1, edge_detection)
    
    printImage(filtered_image_edges, "Outline Image Filtering")
    
    return filtered_image_edges

# Function to calculate PSNR value
def PSNR(original, filtered): 
    mse = np.mean((original - filtered) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 

# Function to calculate MSE value
def MSE(original, filtered):
    err = np.sum((original.astype("float") - filtered.astype("float")) ** 2)
    err /= float(original.shape[0] * original.shape[1])
    return err

# Function to print PSNR and MSE value
def printPSNRnMSE(img, filtered, img_type):
    print(img_type)
    print(f"PSNR value is {PSNR(img, filtered)} dB")
    print(f"MSE value is {MSE(img, filtered)} dB")

# Main function
def main(filename):
    # Input image into RGB colorspace
    img = plt.imread(filename)
    printImage(img, "Original Image")
    img_hsv = rgbToHsv(img)

    #Applying contrast enhancement into HSV Image
    img_hist, img_contr, img_clahe = contrast_enhancement(img_hsv)

    #Applying image filtering into contrast enhance image
    mean_img, med_img, gaussian_img = image_filtering(img_hist)

    #Applying outline image filtering into filtered image
    img_outline = outline_image_filter(gaussian_img)

    #Calculate PSNR and MSE value of processed image
    printPSNRnMSE(img, img_hsv, "HSV Image")
    printPSNRnMSE(img, img_hist, "Histogram Equalization")
    printPSNRnMSE(img, img_contr, "Contrast Stretching")
    printPSNRnMSE(img, img_clahe, "CLAHE")
    printPSNRnMSE(img, mean_img, "Mean Filter")
    printPSNRnMSE(img, med_img, "Median Filter")
    printPSNRnMSE(img, gaussian_img, "Gaussian Filter")
    printPSNRnMSE(img, img_outline, "Outline Image Filter")
    plt.plot([PSNR(img, img_hsv), PSNR(img, img_hist), PSNR(img, img_contr), PSNR(img, img_clahe), PSNR(img, mean_img), PSNR(img, med_img), PSNR(img, gaussian_img), PSNR(img, img_outline)],[1,2,3,4,5,6,7,8])